In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import matplotlib.pyplot as plt
import pickle
import copy
from sklearn.preprocessing import normalize

In [2]:
with open('../../statesToData.pickle', 'rb') as handle:
    statesToData = pickle.load(handle)

In [3]:
all_data_list = []
for i,state in enumerate(statesToData):
    df = statesToData[state]
    all_data_list.append(df)



In [4]:
all_data = pd.concat(all_data_list, axis = 0)

In [14]:
print(all_data.shape)

(781811, 261)


In [5]:
all_data.head()

,CDIVMSAR,CENSUS_D,CENSUS_R,DRIVER,DRVRCNT,ENDTIME,EDUC,GASPRICE,HBHTNRNT,HBHUR,...,PTRANS_AGREE,SPHONE_HIGH,TAB_HIGH,TAXI_HIGH,WALK_HIGH,WALK2SAVE_AGREE,WEBUSE17_HIGH,HHCNTYFP,HHCT,CENSUS_ID
0,53,5,3,1,3,1015,3,228.4,20,T,...,0,1,1,0,0,0,1,191,001301,37191001301
1,53,5,3,1,3,1530,3,228.4,20,T,...,0,1,1,0,0,0,1,191,001301,37191001301
2,53,5,3,1,3,900,3,228.4,20,T,...,0,1,1,0,0,0,1,191,001301,37191001301
3,53,5,3,1,3,2030,3,228.4,20,T,...,0,1,1,0,0,0,1,191,001301,37191001301
4,53,5,3,1,3,900,2,228.4,20,T,...,0,1,1,0,0,0,1,191,001301,37191001301


In [9]:
print(count)

{'37191001301': 53, '37161960300': 27, '37129012105': 203, '37107011400': 44, '37001020501': 11, '37119006302': 30, '37127011300': 34, '37049960500': 98, '37191001500': 7, '37189920900': 54, '37047930800': 70, '37147001303': 183, '37191000700': 48, '37193960400': 76, '37147000301': 109, '37183054211': 31, '37113970400': 61, '37127010300': 114, '37191001800': 61, '37133001700': 26, '37013930900': 44, '37059080500': 26, '37111970200': 81, '37147001302': 46, '37035010600': 23, '37195001600': 27, '37021000400': 22, '37139960100': 17, '37121950200': 52, '37067002002': 4, '37119005812': 59, '37067003404': 2, '37021003102': 22, '37091950100': 64, '37055970300': 53, '37085070402': 19, '37113970200': 52, '37099950500': 46, '37051001400': 50, '37129012006': 83, '37101041001': 12, '37009970700': 64, '37135011102': 12, '37139960501': 58, '37191000401': 107, '37151031304': 13, '37113970700': 47, '37121950400': 106, '37151030504': 53, '37021003104': 3, '37179020601': 4, '37129011701': 137, '37081011

In [16]:
all_data = all_data.groupby('CENSUS_ID').filter(lambda x : (x['CENSUS_ID'].count()>=10).any())

In [26]:
print(all_data.shape)
print('37051002200' in all_data["CENSUS_ID"].values)
print('37129012202' in all_data["CENSUS_ID"].values)

(714943, 261)
False
True


In [27]:
categorical_variables = ["TRIPPURP", "TRPTRANS"]
all_variables = ["CENSUS_ID", "HHVEHCNT", "TRIPPURP", "TRPTRANS"]
labels = all_data.copy(deep=True)[all_variables]

In [28]:
labels.head()


,CENSUS_ID,HHVEHCNT,TRIPPURP,TRPTRANS
0,37191001301,3,0,2
1,37191001301,3,0,2
2,37191001301,3,3,4
3,37191001301,3,3,4
4,37191001301,3,0,2


In [29]:
labels.shape

(714943, 4)

In [30]:
getEncoderList = labels.copy(deep=True)
encoderListAll = list(set(pd.get_dummies(data=getEncoderList[all_variables], columns=categorical_variables).columns))
encoderListAll.sort()
print(len(encoderListAll))
print(encoderListAll)

13
['CENSUS_ID', 'HHVEHCNT', 'TRIPPURP_0', 'TRIPPURP_1', 'TRIPPURP_2', 'TRIPPURP_3', 'TRIPPURP_4', 'TRPTRANS_1', 'TRPTRANS_2', 'TRPTRANS_3', 'TRPTRANS_4', 'TRPTRANS_5', 'TRPTRANS_6']


In [31]:
def one_hot_encode(variables, data_frame):
    data = data_frame.copy()
    #create columns for the indicator variables
    one_hot_encoding = pd.get_dummies(data = data, columns=variables)
    one_hot_encoding = one_hot_encoding.reindex(columns=encoderListAll, fill_value = 0)    
    return one_hot_encoding

In [32]:
encoded_labels = one_hot_encode(categorical_variables, labels)
print(encoded_labels.shape)

(714943, 13)


In [33]:
encoded_labels.head()

,CENSUS_ID,HHVEHCNT,TRIPPURP_0,TRIPPURP_1,TRIPPURP_2,TRIPPURP_3,TRIPPURP_4,TRPTRANS_1,TRPTRANS_2,TRPTRANS_3,TRPTRANS_4,TRPTRANS_5,TRPTRANS_6
0,37191001301,3,1,0,0,0,0,0,1,0,0,0,0
1,37191001301,3,1,0,0,0,0,0,1,0,0,0,0
2,37191001301,3,0,0,0,1,0,0,0,0,1,0,0
3,37191001301,3,0,0,0,1,0,0,0,0,1,0,0
4,37191001301,3,1,0,0,0,0,0,1,0,0,0,0


In [34]:
encoded_avg_labels = encoded_labels.groupby("CENSUS_ID").mean()

In [35]:
encoded_avg_labels.shape

(19898, 12)

In [36]:
encoded_avg_labels.head()

,HHVEHCNT,TRIPPURP_0,TRIPPURP_1,TRIPPURP_2,TRIPPURP_3,TRIPPURP_4,TRPTRANS_1,TRPTRANS_2,TRPTRANS_3,TRPTRANS_4,TRPTRANS_5,TRPTRANS_6
CENSUS_ID,,,,,,,,,,,,
01001020802,2.000000,0.047619,0.000000,0.000000,0.142857,0.809524,0.000,0.666667,0.333333,0.000000,0.0,0.0
01003010300,2.000000,0.176471,0.411765,0.117647,0.000000,0.294118,0.000,0.000000,0.882353,0.117647,0.0,0.0
01003010703,1.954545,0.272727,0.318182,0.000000,0.227273,0.181818,0.000,0.545455,0.363636,0.090909,0.0,0.0
01003011202,3.000000,0.250000,0.625000,0.125000,0.000000,0.000000,0.125,0.250000,0.625000,0.000000,0.0,0.0
01003011401,3.000000,0.250000,0.312500,0.062500,0.125000,0.250000,0.000,0.312500,0.562500,0.125000,0.0,0.0


In [37]:
from sklearn.model_selection import train_test_split
encoded_avg_labels_train, encoded_avg_labels_test = train_test_split(encoded_avg_labels, train_size = 5/6, test_size = 1/6, shuffle = True)

In [38]:
print(encoded_avg_labels_train.shape)
encoded_avg_labels_train.head()

(16581, 12)


,HHVEHCNT,TRIPPURP_0,TRIPPURP_1,TRIPPURP_2,TRIPPURP_3,TRIPPURP_4,TRPTRANS_1,TRPTRANS_2,TRPTRANS_3,TRPTRANS_4,TRPTRANS_5,TRPTRANS_6
CENSUS_ID,,,,,,,,,,,,
05131010301,1.545455,0.000000,0.181818,0.090909,0.272727,0.454545,0.000000,0.000000,0.454545,0.545455,0.0,0.0
48167722800,1.600000,0.100000,0.200000,0.200000,0.100000,0.400000,0.000000,1.000000,0.000000,0.000000,0.0,0.0
34033020300,3.000000,0.181818,0.181818,0.136364,0.045455,0.454545,0.090909,0.636364,0.000000,0.272727,0.0,0.0
48491020313,2.444444,0.185185,0.185185,0.111111,0.129630,0.388889,0.000000,0.388889,0.407407,0.203704,0.0,0.0
36111954800,1.484848,0.166667,0.166667,0.121212,0.060606,0.484848,0.181818,0.636364,0.136364,0.045455,0.0,0.0


In [39]:
print(encoded_avg_labels_test.shape)
encoded_avg_labels_test.head()

(3317, 12)


,HHVEHCNT,TRIPPURP_0,TRIPPURP_1,TRIPPURP_2,TRIPPURP_3,TRIPPURP_4,TRPTRANS_1,TRPTRANS_2,TRPTRANS_3,TRPTRANS_4,TRPTRANS_5,TRPTRANS_6
CENSUS_ID,,,,,,,,,,,,
13215011100,2.211538,0.250000,0.192308,0.076923,0.096154,0.384615,0.076923,0.307692,0.057692,0.519231,0.000000,0.038462
48157671002,1.818182,0.136364,0.272727,0.045455,0.159091,0.386364,0.000000,0.545455,0.363636,0.068182,0.022727,0.000000
06071010020,0.666667,0.333333,0.222222,0.111111,0.111111,0.222222,0.111111,0.222222,0.000000,0.444444,0.222222,0.000000
48201423201,2.000000,0.047619,0.190476,0.190476,0.047619,0.523810,0.190476,0.571429,0.238095,0.000000,0.000000,0.000000
36111951300,2.269565,0.295652,0.200000,0.104348,0.121739,0.278261,0.043478,0.321739,0.408696,0.156522,0.052174,0.017391


In [40]:
encoded_avg_labels_train.to_csv("encoded_avg_less_labels_train.csv")

In [41]:
encoded_avg_labels_test.to_csv("encoded_avg_less_labels_test.csv")